In [12]:
import os
import cv2
import json
import shutil
import scipy.io as sio
import numpy as np
from tiseg.datasets.utils import colorize_seg_map

/home/zhangye/.local/lib/python3.7/site-packages/tqdm-4.65.0-py3.7.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
with open("./data/consep/train_val_test.json", 'r') as file:
        data_list = json.load(file)

data_list.keys()
train_data = data_list["train"]
valid_data = data_list["val"]
test_data = data_list["test"]

In [ ]:
file_path = f"./data/consep/test.txt"
f = open(file_path, "a")
for i in os.listdir("./data/consep/test/Images"):
    i = i.split(".")[0]
    print(i)
    f.write(str(i)+'\n')
f.close()

# 生成edge distance map

In [48]:
stage = "valid"
inst_lab_path = f"/data4/zhangye/Tissue-Image-Segmentation-master/data/consep/{stage}"

data_list = [f for f in os.listdir(inst_lab_path) if f.endswith("instance.npy")]
for f_path in data_list:
    f_name = f_path.split(".")[0]
    # print(f_name)

    inst_lab = np.load(f"{inst_lab_path}/{f_path}")
    edge_dis = np.zeros_like(inst_lab)#.reshape(256,256,1)
    inst_id = list(np.unique(inst_lab))
    inst_id.pop(0)
    for inst in inst_id:
        inst_id_mask = inst_lab==inst
        bound = morphology.dilation(
                    inst_id_mask, selem=morphology.selem.diamond(3)) & (
                        ~morphology.erosion(inst_id_mask, selem=morphology.selem.diamond(1)))
        edge_dis[bound >0] = 1
    edge_dis = 1-edge_dis.astype(np.uint8)

    edge_dis= cv2.distanceTransform(edge_dis, distanceType=cv2.DIST_L2, maskSize=3)
    # normalized_dist = cv2.normalize(edge_dis, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # edge_dis = np.clip(edge_dis,0,20 )
    edge_dis[inst_lab == 0] = - edge_dis[inst_lab == 0]
    edge_dis = np.clip(edge_dis, -40, 20)
    edge_dis[edge_dis<0] = -edge_dis[edge_dis<0]/(edge_dis[edge_dis<0].min())
    # print(edge_dis[edge_dis>0])
    if edge_dis[edge_dis>0].sum() == 0:
        print(f_name)
    else:
        edge_dis[edge_dis>0] = edge_dis[edge_dis>0]/(edge_dis[edge_dis>0].max())

    edge_dis = edge_dis.reshape(256,256,1)

    dis_map = np.zeros((256,256,1))

    for inst in inst_id:
        # print("inst", inst)
        inst_id_mask = np.array(inst_lab==inst, np.uint8)
        # print(inst_id_mask.shape)
        # plt.imshow(inst_id_mask)
        # plt.show()
        inst_box = get_bounding_box(inst_id_mask)
        inst_box[0] -= 2
        inst_box[2] -= 2
        inst_box[1] += 2
        inst_box[3] += 2
        inst_map = inst_id_mask[inst_box[0] : inst_box[1], inst_box[2] : inst_box[3]]
        if inst_map.shape[0] < 2 or inst_map.shape[1] < 2:
            continue
        # plt.imshow(inst_map)
        # plt.show()

        inst_com = list(measurements.center_of_mass(inst_map))
        inst_com[0] = int(inst_com[0] + 0.5)
        inst_com[1] = int(inst_com[1] + 0.5)


        centroid_y ,centroid_x = inst_com[0], inst_com[1]
        height, width = inst_map.shape
        y, x = np.indices((height, width))
        dx, dy = x-centroid_x, y-centroid_y

        theta = np.arctan2(dy, dx)
        rho = np.sqrt(dx**2 + dy**2)

        if np.min(theta) < 0:
            theta[theta<0] /= -np.amin(theta[theta<0])
        # if np.min(rho) < 0:
        #     rho[rho<0] /= -np.amin(rho[rho<0])

        if np.max(theta) > 0:
            theta[theta>0] /= np.amax(theta[theta>0])
        if np.max(rho) > 0:
            rho[rho>0] /= np.amax(rho[rho>0])

        # dis_map_box = dis_map[inst_box[0] : inst_box[1], inst_box[2] : inst_box[3],0]
        # dis_map_box[inst_map>0] = theta[inst_map>0]
        angel_map_box = dis_map[inst_box[0] : inst_box[1], inst_box[2] : inst_box[3],0]
        angel_map_box[inst_map>0] = rho[inst_map>0]
    
    dis_sup_map = np.concatenate([edge_dis, dis_map], axis=2)
    

    np.save(f"{inst_lab_path}/{f_name}_sup_dis.npy", dis_sup_map)
    # edge_dis[inst_lab == 0] = - edge_dis[inst_lab == 0]
    # cv2.imwrite(f"{inst_lab_path}/{f_name}_dis.png",edge_dis*255)
    



test_12_17_instance
test_13_6_instance
test_12_23_instance
test_12_6_instance


In [141]:
inst_lab = np.load("/data4/zhangye/Tissue-Image-Segmentation-master/data/consep/train/train_11_20_instance.npy")
edge_dis = np.zeros_like(inst_lab)#.reshape(256,256,1)
inst_id = list(np.unique(inst_lab))
inst_id.pop(0)
for inst in inst_id:
    inst_id_mask = inst_lab==inst
    bound = morphology.dilation(
                inst_id_mask, selem=morphology.selem.diamond(3)) & (
                    ~morphology.erosion(inst_id_mask, selem=morphology.selem.diamond(1)))
    edge_dis[bound >0] = 1
edge_dis = 1-edge_dis.astype(np.uint8)

edge_dis= cv2.distanceTransform(edge_dis, distanceType=cv2.DIST_L2, maskSize=5)
edge_dis[inst_lab == 0] = - edge_dis[inst_lab == 0]
edge_dis = np.clip(edge_dis, -40, 20)
edge_dis[edge_dis<0] = -edge_dis[edge_dis<0]/(edge_dis[edge_dis<0].min())
# normalized_dist = cv2.normalize(edge_dis, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)


In [2]:
def get_bounding_box(img):
    """Get bounding box coordinate information."""
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    # due to python indexing, need to add 1 to max
    # else accessing will be 1px in the box, not out
    rmax += 1
    cmax += 1
    return [rmin, rmax, cmin, cmax]

def gen_instance_hv_map(ann, crop_shape=None):
    """Input annotation must be of original shape.
    
    The map is calculated only for instances within the crop portion
    but based on the original shape in original image.

    Perform following operation:
    Obtain the horizontal and vertical distance maps for each
    nuclear instance.

    """
    orig_ann = ann.copy()  # instance ID map
#     fixed_ann = fix_mirror_padding(orig_ann)
    # re-cropping with fixed instance id map
#     crop_ann = cropping_center(fixed_ann, crop_shape)
    # TODO: deal with 1 label warning
    
#     crop_ann = morph.remove_small_objects(crop_ann, min_size=30)

    x_map = np.zeros(orig_ann.shape[:2], dtype=np.float32)
    y_map = np.zeros(orig_ann.shape[:2], dtype=np.float32)

    inst_list = list(np.unique(orig_ann))
    inst_list.remove(0)  # 0 is background
    for inst_id in inst_list:
#         print("inst id", inst_id)
        inst_map = np.array(orig_ann == inst_id, np.uint8)
        inst_box = get_bounding_box(inst_map)

        # expand the box by 2px
        # Because we first pad the ann at line 207, the bboxes
        # will remain valid after expansion
        inst_box[0] -= 2
        inst_box[2] -= 2
        inst_box[1] += 2
        inst_box[3] += 2

        inst_map = inst_map[inst_box[0] : inst_box[1], inst_box[2] : inst_box[3]]

        if inst_map.shape[0] < 2 or inst_map.shape[1] < 2:
            continue

        # instance center of mass, rounded to nearest pixel
        inst_com = list(measurements.center_of_mass(inst_map))

        inst_com[0] = int(inst_com[0] + 0.5)
        inst_com[1] = int(inst_com[1] + 0.5)

        inst_x_range = np.arange(1, inst_map.shape[1] + 1)
        inst_y_range = np.arange(1, inst_map.shape[0] + 1)
        # shifting center of pixels grid to instance center of mass
        inst_x_range -= inst_com[1]
        inst_y_range -= inst_com[0]

        inst_x, inst_y = np.meshgrid(inst_x_range, inst_y_range)

        # remove coord outside of instance
        inst_x[inst_map == 0] = 0
        inst_y[inst_map == 0] = 0
        inst_x = inst_x.astype("float32")
        inst_y = inst_y.astype("float32")

        # normalize min into -1 scale
        if np.min(inst_x) < 0:
            inst_x[inst_x < 0] /= -np.amin(inst_x[inst_x < 0])
        if np.min(inst_y) < 0:
            inst_y[inst_y < 0] /= -np.amin(inst_y[inst_y < 0])
        # normalize max into +1 scale
        if np.max(inst_x) > 0:
            inst_x[inst_x > 0] /= np.amax(inst_x[inst_x > 0])
        if np.max(inst_y) > 0:
            inst_y[inst_y > 0] /= np.amax(inst_y[inst_y > 0])

        ####
        x_map_box = x_map[inst_box[0] : inst_box[1], inst_box[2] : inst_box[3]]
        x_map_box[inst_map > 0] = inst_x[inst_map > 0]

        y_map_box = y_map[inst_box[0] : inst_box[1], inst_box[2] : inst_box[3]]
        y_map_box[inst_map > 0] = inst_y[inst_map > 0]
    hv_map = np.dstack([x_map, y_map])
    
    return hv_map

In [4]:
from tqdm import tqdm

stage = "test"
inst_lab_path = f"/data4/zhangye/Tissue-Image-Segmentation-master/data/kumar/{stage}"

data_list = [f for f in os.listdir(inst_lab_path) if f.endswith("instance.npy")]
for f_path in tqdm(data_list):
    
    f_name = f_path.split(".")[0]
    # print("f name", f_name)
    inst_lab = np.load(f"{inst_lab_path}/{f_path}")
    
    se_map = np.zeros_like(inst_lab)
    bg_dis_map = np.zeros_like(inst_lab)

    inst_id = list(np.unique(inst_lab))
    inst_id.pop(0)
    for inst in inst_id:
        inst_id_mask = np.array(inst_lab==inst, np.uint8)

        inst_box = get_bounding_box(inst_id_mask)
        inst_box[0] -= 2
        inst_box[2] -= 2
        inst_box[1] += 2
        inst_box[3] += 2
        if inst_box[0]<0:
            inst_box[0] = 0
        if inst_box[2]<0:
            inst_box[2] = 0

        inst_map = inst_id_mask[inst_box[0] : inst_box[1], inst_box[2] : inst_box[3]]
        if inst_map.shape[0] < 2 or inst_map.shape[1] < 2:
            continue

        

        """SE_map"""
        bound = morphology.dilation(
                    inst_map, selem=morphology.selem.diamond(3)) & (
                        ~morphology.erosion(inst_map, selem=morphology.selem.diamond(1)))
        bound_ = morphology.dilation(
                    inst_id_mask, selem=morphology.selem.diamond(3)) & (
                        ~morphology.erosion(inst_id_mask, selem=morphology.selem.diamond(1)))
        inst_edge = 1-(bound>0).astype(np.uint8)
        inst_dis = cv2.distanceTransform(inst_edge, distanceType=cv2.DIST_L2, maskSize=3)
        inst_dis /= inst_dis.max()
        se_map_box = se_map[inst_box[0] : inst_box[1], inst_box[2] : inst_box[3]]
        se_map_box[inst_map>0] = inst_dis[inst_map>0]
        bg_dis_map[bound_>0] = 1
    
    bg_dis = 1- bg_dis_map.astype(np.uint8)
    
    bg_dis= cv2.distanceTransform(bg_dis, distanceType=cv2.DIST_L2, maskSize=5)
    bg_dis /= bg_dis.max()
    # plt.imshow(bg_dis)
    # plt.show()
    se_map[inst_lab==0] = -bg_dis[inst_lab==0]
    np.save(f"{inst_lab_path}/{f_name}_instance_se.npy", se_map)

    """HV-map"""
    hv_map = gen_instance_hv_map(inst_lab)
    np.save(f"{inst_lab_path}/{f_name}_instance_hv.npy", hv_map)

    """Dir-map"""
    dir_map = get_dir_from_inst(inst_lab, 8)
    np.save(f"{inst_lab_path}/{f_name}_instance_dir.npy", dir_map)
 
       


100%|██████████| 294/294 [05:18<00:00,  1.08s/it]
